In [1]:
import sqlite3 
import data_manager as dm
import datetime as dt
import pandas as pd
from models import *
# from sqlalchemy import create_engine

In [2]:
# Создать новый файл БД (или открыть сущетвующий файл)
db_name = 'D:/YandexDisk/it_Projects/py_databases/paymens.db'
con = sqlite3.connect(db_name)

# Создать таблицы базы данных
dm.create_new_base()

In [69]:
# Загрузить CSV файл в DataFrame
filename = 'D:/IDE_prj/project_KnowledgeBase/personal_expenses/data_input/tst.csv'
df = pd.read_csv(filename, sep=';', encoding='cp1251')

# Создать столбец "date" (перобразовать тип поля текст в дату)
df["date"] = pd.to_datetime(df["Дата операции"], dayfirst=True)

# Создать стобцы "receipt" (Приход) и "expense" (Расход)
mask_receipt = df["Сумма операции в валюте карты"] > 0
mask_expense = df["Сумма операции в валюте карты"] < 0
df['receipt'] = df[mask_receipt]["Сумма операции в валюте карты"]
df['expense'] = -df[mask_expense]["Сумма операции в валюте карты"]
d1 = df.date.min().date()
d2 = (df.date.max()+pd.offsets.Day(1)).date()
# [df.receipt.sum(), df.expense.sum(), d1, d2]

# df = df.head(3)
df.head(3)
# df.tail(3)
# df.info()


,Дата операции,Дата обработки,Название операции,Сумма операции в валюте карты,Комиссия в валюте карты,Сумма операции в валюте операции,Валюта операции,Категория,Место операции,date,receipt,expense
0,05.03.2022 12:57,05.03.2022,NaN,360.0,0.0,360.0,RUR,NaN,NaN,2022-03-05 12:57:00,360.0,NaN
1,05.03.2022 15:12,09.03.2022,Покупка RUS SEVASTOPOL SUPERTIAZH,-287.0,0.0,-287.0,RUR,"Ресторан, кафе, бар",NaN,2022-03-05 15:12:00,NaN,287.0
2,05.03.2022 16:32,09.03.2022,Покупка RUS SEVASTOPOL 7M BEZ TCEN,-88.8,0.0,-88.8,RUR,Супермаркеты,NaN,2022-03-05 16:32:00,NaN,88.8


In [8]:
# Создать запись карты РНКБ
card = BankCard(number='2200020233972043', name='Карта РНКБ')
card.save()


1

In [70]:
def get_banc_card():
    sel = BankCard.select().where(BankCard.number == '2200020233972043')
    if len(sel) != 1:
        print("В базе данных не найдена банковская карта с запрашиваемым именем")
        return
    bank_card = sel.get()
    print(f'{bank_card.name}: {bank_card.number}')
    return bank_card

def get_dt_from_pandas(value):
    return dt.datetime(
        value.year,
        value.month,
        value.day,
        value.hour,
        value.minute,
        value.second
    )

bank_card = get_banc_card()

payments = [PamentBankCard(
        card_id      = bank_card, 
        date         = get_dt_from_pandas(rec.date),
        date_account = dt.datetime.now(), #['Дата обработки'],
        name         = rec['Название операции'],
        amount       = rec['Сумма операции в валюте карты'],
        category     = rec['Категория'],
        place        = rec['Место операции'],
    ) for i,rec in df.iterrows()
]

# for rec in payments:
#     print(rec.card_id, rec.date, rec.date_account, rec.name, rec.amount, rec.category, rec.place, type(rec.date))

PamentBankCard.bulk_create(payments, batch_size=999)
# payment.bu

Карта РНКБ: 2200020233972043
2 2022-03-05 12:57:00 2022-04-26 10:32:53.318072 nan 360.0 nan nan <class 'datetime.datetime'>
2 2022-03-05 15:12:00 2022-04-26 10:32:53.318072 Покупка RUS SEVASTOPOL SUPERTIAZH -287.0 Ресторан, кафе, бар nan <class 'datetime.datetime'>
2 2022-03-05 16:32:00 2022-04-26 10:32:53.318072 Покупка RUS SEVASTOPOL 7M BEZ TCEN -88.8 Супермаркеты nan <class 'datetime.datetime'>
2 2022-03-07 15:26:00 2022-04-26 10:32:53.319080 nan 200.0 nan nan <class 'datetime.datetime'>
2 2022-03-07 15:38:00 2022-04-26 10:32:53.319080 Покупка RUS SEVASTOPOL MAGAZIN VIKTORIIA -100.0 Супермаркеты nan <class 'datetime.datetime'>
2 2022-03-08 07:40:00 2022-04-26 10:32:53.319080 nan 4500.0 nan nan <class 'datetime.datetime'>
2 2022-03-08 07:41:00 2022-04-26 10:32:53.319080 Выдача наличных RUS SEVASTOPOL ATM-8509 -4500.0 Снятие наличных nan <class 'datetime.datetime'>
2 2022-03-10 14:57:00 2022-04-26 10:32:53.319080 Пополнение через POS-терминал 1300000.0 nan nan <class 'datetime.datetim

In [66]:
# Получить объект банковской карты

def test():
    sel = BankCard.select().where(BankCard.number == '2200020233972043')
    if len(sel) != 1:
        print("В базе данных не найдена банковская карта с запрашиваемым именем")
        return
    bank_card = sel.get()
    print(bank_card.name)
    
test()

Карта РНКБ (Kovex)


In [74]:
def print_records(d1, d2):


    # Выдать даты по карте за период
    sel = PamentBankCard.select().where(
        (PamentBankCard.card_id == bank_card) 
        & (PamentBankCard.date >= d1) 
        & (PamentBankCard.date < d2)
        )

    print(f'Количество записей: {len(sel)}')

    for rec in sel:
        print(rec.date)

print_records(
    dt.datetime(2022,3,10), 
    dt.datetime(2022,3,16))

Количество записей: 9
2022-03-10 14:57:00
2022-03-12 16:16:00
2022-03-12 16:18:00
2022-03-13 14:50:00
2022-03-13 21:41:00
2022-03-13 21:50:00
2022-03-15 15:34:00
2022-03-15 18:38:00
2022-03-15 20:38:00


In [80]:
def delete_records(card, d1, d2):
    # Выдать даты по карте за период
    sel = PamentBankCard.select().where(
        (PamentBankCard.card_id == card) 
        & (PamentBankCard.date >= d1) 
        & (PamentBankCard.date < d2)
        )
    with db.atomic() as transaction:  # transaction.rollback()
        for rec in sel:
            rec.delete_instance()

delete_records(
    bank_card,
    dt.datetime(2022,3,1), 
    dt.datetime(2022,3,6))

In [81]:
print_records(
    dt.datetime(2022,3,1), 
    dt.datetime(2022,3,16))

Количество записей: 4
2022-03-07 15:26:00
2022-03-07 15:38:00
2022-03-08 07:40:00
2022-03-08 07:41:00


In [ ]:
import tkinter.filedialog as fd
filetypes = (("CSV", "*.csv"),
                ("Сбербанк PDF", "*.pdf"),
                ("Любой", "*"))                
filename = fd.askopenfilename(title="Открыть файл", #initialdir=os.getcwd(), #"/",
                                filetypes=filetypes)


In [1]:
import os
os.path.splitext(filename)[1].lower()


NameError: name 'filename' is not defined